# Camera Calibration

Below is a python script that allows you to compute the intrinsic and extrinsic camera parameters based on a set of checkerboard images. This code utilizes functions provided by the OpenCV-Python package.

Follow the instructions written below to extract the necessary camera calibration parameters.

In [ ]:
# Code adapted from openCV documentation example:
# https://opencv24-python-tutorials.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_calibration/py_calibration.html
# accessed 11 Jul 22


# https://docs.pixycam.com/wiki/doku.php?id=wiki:v2:installing_pixymon_on_linux pixymon installation guide

# Import required modules
import cv2
import numpy as np
import os
from tqdm import tqdm

Modify the below cell prior to running the next cells

In [ ]:
# Extract list of paths to checkerboard images
# Modify accordingly
current_path = os.path.dirname(os.path.abspath(" "))
image_path = os.path.join(current_path, "images/")
image_names = os.listdir(image_path)
num_images = len(image_names)
# In some cases, slight blurring of the image may aid checkerboard detection
# Feel free to attempt different filter sizes for blurring before settling on a specific filter size
filt_size = [1, 2]
if(num_images == 0):
    print("No images found in the specified directory")

**Make sure that at least 10 images are analyzed successfully.** </br>
Avoiding glare can help improve checkerboard detection.

The `calibrateCamera()` function returns the camera matrix, distortion coefficients, rotation and translation vectors in the following form:
$$ \mathrm{camera \: matrix} = 
                \begin{bmatrix} 
                    \alpha_u & 0   & u_0 \\ 
                    0   & \alpha_v & v_0 \\
                    0   & 0   & 1 
                \end{bmatrix}$$
$$ \mathrm{distortion \: coefficients} = \begin{bmatrix} k_1 & k_2  \end{bmatrix}$$

$\alpha_u$ and $\alpha_v$ are the camera focal lengths in pixel.

$k_1$ and $k_2$ are the radial distortion coefficients.

$u_0$ and $v_0$ are the optical centers in pixel.

Scale factor: It is used to scale pixy-image such that it fits the image from the Python calibration procedure

In [ ]:
# We are using an 8x10 checkerboard image
# The open-cv findChessboardCorners() only detects internal corners
DIMS = (7, 9)

# Modify this variable to display the analyzed image
# If TRUE, press any key to move on to the next image
# If you wish to stop examining images, you can press 'a' to analyze the remaining images without displaying them
show_images = True
key = 0

# stop criteria for subpix coordinate iteration loop
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

for filt in filt_size:
    count = 0

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((DIMS[0] * DIMS[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0 : DIMS[0], 0 : DIMS[1]].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d point in real world space
    imgpoints = []  # 2d points in image plane.

    for idx in tqdm(range(num_images)):

        filename = image_path + image_names[idx]
        image = cv2.imread(filename)

        h, w, c = image.shape

        # blurring the image with a blur window of size (filt x filt)
        blur = cv2.blur(image, (filt, filt))

        # converting into grayscale for chessboard corner detection
        grayColor = cv2.cvtColor(blur, cv2.COLOR_BGR2GRAY)
        gray_display = cv2.cvtColor(grayColor, cv2.COLOR_GRAY2BGR)

        # Find the chess board corners
        ret, corners = cv2.findChessboardCorners(
            grayColor, DIMS, cv2.CALIB_CB_FAST_CHECK
        )

        # If found, add object points, image points (after refining them)
        if ret == True:
            count += 1
            objpoints.append(objp)

            corners2 = cv2.cornerSubPix(
                grayColor, corners, (11, 11), (-1, -1), criteria
            )

            imgpoints.append(corners2)

            # Draw and display the corners
            image = cv2.drawChessboardCorners(image, DIMS, corners2, ret)

        if show_images:

            both_images = np.hstack((gray_display, image))
            cv2.putText(
                both_images,
                "Press any key to see next image",
                (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 0, 255),
                2,
                2,
            )
            cv2.putText(
                both_images,
                "Press 'a' to stop inspecting images individually",
                (10, 65),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (0, 0, 255),
                2,
                2,
            )
            cv2.imshow(filename, both_images)
            key = cv2.waitKey(0)

            if key == 97:  # equals to a
                show_images = False
            cv2.destroyAllWindows()
            cv2.waitKey(1)

    ## outputs
    print(
        f"Filter size: {filt}\n{count} out of {num_images} images analyzed successfully."
    )
    print(f"Image height = {h}, width={w}")
    print(f"Scale factor in x = {w/315:.2f}, and in y = {h/207:.2f}")
    print(f"calibration_image_scale factor = {(h/207 + w/315)/2:.2f}")

    # camera calibration, the flags ensure that a radial distortion model with 2 coefficients is used for the calibration
    ret, mtx, dist, r_vecs, t_vecs = cv2.calibrateCamera(
        objpoints,
        imgpoints,
        grayColor.shape[::-1],
        None,
        None,
        flags=cv2.CALIB_FIX_K3 + cv2.CALIB_ZERO_TANGENT_DIST,
    )

    # Printing only the values necessary for our camera calibration
    print(
        f"Radial distortion coefficients k1 = {dist[0][0]:.4f} and k2 = {dist[0][1]:.4f}"
    )
    print(f"Focal lengh in pixel: {(mtx[0][0]+mtx[1][1])/2:.4f}")
    print(f"Optical centers in pixel u_0 = {mtx[0][2]:.4f} and v_0 = {mtx[1][2]:.4f}")

    # Calculation of the reprojection error
    tot_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i], r_vecs[i], t_vecs[i], mtx, dist)
        error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
        tot_error += error

    print(f"Reprojection error: {tot_error/len(objpoints):.4f}")